# SESAMME Cube Maker Demo

This notebook illustrates how to make a FITS data cube containing a suite of simple stellar population models (SSPs) from individual SSP files, which can be passed to SESAMME to perform full spectral fitting of star clusters. In this notebook, you can also optionally create an ascii table containing the number of hydrogen-ionizing photons per second Q(HI) emitted by each SSP in your model grid.




By the end of this tutorial, you will have a working example of how to:
1. Create an SSP model cube, trimmed in age, metallicity, or wavelength to fit your science needs; 
2. Create an array of Q(HI) values for your suite of spectra.

For an example of how to use the model cube to characterize your star cluster, please see the accompanying notebook `SESAMME Demo`. For illustrations of other steps in the modeling process (for example, how we rebinned our high-resolution spectra to the 1 A resolution of BPASS, or one way to create mock noise-added spectra of extragalactic star clusters), please see the accompanying notebook `Resampling Demo`.

## Imports

In [19]:
### File handling tools
import os, glob
from astropy.io import fits

### Manipulating arrays
import numpy as np

## Making a model cube with BPASS v2.3

Individual files containing a model SSP (at some age and metallicity) are read in, cropped to a user-defined wavelength range, and then stacked to form a FITS cube. Each FITS extension in the final cube is a BinTable object for spectra with a given metallicity, and each column within the table is the SED of the stellar population at the specified age.

The cells below sets up most of the important information to be contained in the model cube. This includes:
- The age and metallicity grid
- The file names you want to collate into the cube, plus their parent directory
- The desired wavelength bounds of your model cube, if the parent spectra have more wavelength coverage than is necessary

In this example, we show how to manually specify which files from a given SSP suite to include in the cube. More elegant ways to create the array of file names using, e.g., `glob` will be documented later. 

For this example we use the BPASS v2.3 models with binary stars, the 135_300 IMF, and solar $\alpha$-element abundances (see Byrne+ 2022 for details).

In [16]:
# Define the age bins, which will double as column names
ages = np.array([str( round(np.log10(10**(6+0.1*(n-2))), 1) ) for n in range(2,53)])

# Define the metallicity bins, which will double as FITS extension names
mets = ['Z001', 'Z002', 'Z003', 'Z004', 'Z006', 'Z008', 'Z010', 'Z014', 'Z020', 'Z030', 'Z040', 'Zem4', 'Zem5']
met_values = [0.001, 0.002, 0.003, 0.004, 0.006, 0.008, 0.01, 0.014, 0.02, 0.03, 0.04, 1e-4, 1e-5]

# Names for the individual spectrum files to be read in
spec_filenames = ['spectra-bin-imf135_300.a+00.z001',  
'spectra-bin-imf135_300.a+00.z002',  
'spectra-bin-imf135_300.a+00.z003',  
'spectra-bin-imf135_300.a+00.z004',  
'spectra-bin-imf135_300.a+00.z006',  
'spectra-bin-imf135_300.a+00.z008',  
'spectra-bin-imf135_300.a+00.z010',
'spectra-bin-imf135_300.a+00.z014',           
'spectra-bin-imf135_300.a+00.z020',
'spectra-bin-imf135_300.a+00.z030',
'spectra-bin-imf135_300.a+00.z040',
'spectra-bin-imf135_300.a+00.zem4',
'spectra-bin-imf135_300.a+00.zem5']

# Specify the desired min and max wavelength (in Angstroms) for the model cubes
min_wl = 912
max_wl = 3000

# Set the column names for all extensions in the model cube
spec_colnames = ['WL'] + [str(i) for i in ages]

# Specify the input path for the stellar pop. model cube
spec_in_path = "/path/to/file/"

# Specify the output path for the stellar pop. model cube
spec_out_path = "/path/to/file/"

Now we iterate over the individual files, trim them down to the desired wavelength range, and combine them into a multi-extension FITS file. **Because the file names and metallicity labels are zipped together, please ensure that your array of file names and array of Z values are given in the same order.**

Some temporary files will be created in the process. The output will be a FITS file with 14 HDUs - an empty Primary and 13 BinTable extensions labelled by metallicity.

In [17]:
# Create primary HDU and HDUList
primary_hdu = fits.PrimaryHDU()
stellar_list = fits.HDUList([primary_hdu])

for file, metal in zip(spec_filenames, mets):
    spec = Table.read(spec_in_path + file + ".dat", format = 'ascii', names = spec_colnames)
    
    # Writes the wavelength-trimmed version of the SSP to a temporary file with the prefix "short_"
    spec[min_wl - 1 : max_wl].write(spec_in_path + "short_" + file+".fits", overwrite=True, format='fits')
    
    # Read in the temporary file "short_*" and assign it to a new HDU, which is appended to the final cube object
    fitsspec = fits.open(spec_in_path + "short_" + file + ".fits")
    fitsspec[1].header['EXTNAME'] = metal
    stellar_list.append(fitsspec[1])
    
    # Remove the temporary file
    os.remove(spec_in_path + "short_" + file+".fits")
    
# Write the complete cube to file
stellar_list.writeto(spec_out_path + "Demo_Met_Table.fits", overwrite=True)

You now have a multi-extension FITS cube containing your SSP model suite which is compatible with SESAMME. Suppose now that you also want to include nebular continuum emission in your modeling, which is not included in the BPASS SSPs. The cells below allow you to create a grid of ionizing outputs per SSP, which SESAMME can use to generate and add a nebular continuum component to its modeling process.

In the case of BPASS models, each file contains the Q(HI) values for an SSP of the named metallicity at each time step in the BPASS grid.

In [20]:
# Names for the individual N_ion files to be read in
nion_filenames = ['ionizing-bin-imf135_300.a+00.z001',
                  'ionizing-bin-imf135_300.a+00.z002',
                  'ionizing-bin-imf135_300.a+00.z003',
                  'ionizing-bin-imf135_300.a+00.z004',
                  'ionizing-bin-imf135_300.a+00.z006',
                  'ionizing-bin-imf135_300.a+00.z008',
                  'ionizing-bin-imf135_300.a+00.z010',
                  'ionizing-bin-imf135_300.a+00.z014',
                  'ionizing-bin-imf135_300.a+00.z020',
                  'ionizing-bin-imf135_300.a+00.z030',
                  'ionizing-bin-imf135_300.a+00.z040',
                  'ionizing-bin-imf135_300.a+00.zem4',
                  'ionizing-bin-imf135_300.a+00.zem5']

# Specify the input path for the stellar pop. ionizing output
nion_in_path = "/Users/lojones/Documents/BPASS/bpass_v2.3.a+00/"
# nion_in_path = "/path/to/file/"

# Specify the output path for the stellar pop. ionizing output
nion_out_path = "/Users/lojones/Documents/stellaz/Sphinx/"
# nion_out_path = "/path/to/file/"

# Set the column names for the table of Q(HI) values, one row per metallicity
nion_colnames = ['Z'] + [str(x) for x in ages]

Now we iterate over the individual ionizing-flux files and combine them into a simple ascii array. **Because the file names and metallicity labels are zipped together, please ensure that your array of file names and array of Z values are given in the same order.**

The output will be a `*.txt` file containing a 2D array of the log(Q(HI)) values. Each row is a different metallicity, and each column is a different time step in the BPASS grid.

In [21]:
### Iterate over individual ionizing output files for the assumed IMF/abundance ratio, combining them
### into a single ascii table with one metallicity per row.

nion_table = Table(names = nion_colnames, dtype = [str] + [float]*51)

for file, metal in zip(nion_filenames, mets):
    q_table = Table.read(nion_in_path+file+".dat", format='ascii')
    
    nion_table.add_row(np.hstack( ([metal], np.r_[q_table['col2']]) ))
    
nion_table.write(nion_out_path+"Demo_Q_Table.txt", format='ascii', names=nion_colnames, overwrite=True)

To confirm that the files we just made have the format that `SESAMME` expects, we can load them in using the functions `sesamme.models.load_ssp_cube()` and `sesamme.models.load_ionization_table()`.

In [ ]:
modelcube = models.load_ssp_cube("/path/to/cube_file.fits")

iontable = models.load_ionization_table("/path/to/qtable_file.txt")

Congrats, you've successfully created all of the necessary models for your SESAMME run! 